In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import torch
import test_fn

In [107]:
import importlib
importlib.reload(test_fn)

<module 'test_fn' from 'C:\\Users\\Andrew\\OneDrive\\Documents\\HesBO\\baseline\\test_fn.py'>

# Draw sample inputs

In [108]:
n = 100
low_dim = 50
fn_dim = 200
fn = test_fn.Ackley()

In [109]:
x = torch.rand((n,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

In [8]:
X[torch.topk(y, 3, largest = False).indices,:].shape

torch.Size([3, 200])

In [9]:
X.shape

torch.Size([100, 200])

In [110]:
fn = test_fn.Levy(10)

# Algorithm 1: Shift subspace to best performing points

In [10]:
importlib.reload(test_fn)
fn = test_fn.Ackley()
x = torch.rand((100,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

# Find best performing points (Alt: Find best performing and nearest neighbors)
best_performers_indices = torch.topk(y, 1, largest = False).indices
best_performers = X[best_performers_indices]

# Update
A = best_performers

In [11]:
A.shape

torch.Size([1, 200])

In [12]:
torch.concat([A, torch.rand((low_dim - 1, fn_dim))], dim = 0).shape

torch.Size([50, 200])

In [29]:
def learn_subspace_best_performers(fn, low_dim, fn_dim, num_best = 1, samples = 200, max_iter = 1000, trace = False):
    best_found = 1e10
    A = torch.rand((low_dim, fn_dim))
    first_run = True
    
    for i in range(max_iter):
        # Evaluate
        x = torch.randn((samples,low_dim))
        X = x@A  
        y = fn.evaluate(X)

        # Find best performing points (Alt: Find best performing and nearest neighbors)
        best_performers_indices = torch.topk(y, num_best, largest = False).indices
        best_performers = X[best_performers_indices]
        
        # Standardize
#         best_performers = best_performers - torch.mean(best_performers, dim = 1, keepdim = True)
#         best_performers = best_performers/torch.std(best_performers, dim = 1, keepdim = True)

        if trace:
            print(best_performers)

        # Update
        A = torch.concat([best_performers, torch.randn((low_dim - num_best, fn_dim))], dim = 0)

        if min(y) < best_found:
            best_found = min(y)

        if i%int(max_iter/10) == 0:
            print(f"Min/best found y is:{min(y):.2f}, {best_found:.2f}")

In [31]:
learn_subspace_best_performers(fn, low_dim = 10, fn_dim = 200, num_best = 1, trace = True)

tensor([[ 0.1991, -0.4369,  0.3557,  0.4689,  0.2449,  0.0554, -0.6094,  0.5757,
          0.2317,  0.1278, -0.0636, -0.2051, -0.5939, -0.2182, -1.0626,  0.0669,
         -0.1695, -0.4971, -0.8586,  0.5917, -0.2759,  0.3744, -0.4142, -0.8023,
          0.1634, -0.7632, -0.1808,  0.3815, -0.3715, -0.1702, -0.2651, -0.2371,
          0.1869, -0.6500,  0.3171,  0.3074, -0.0423,  0.7760, -0.2588, -0.4214,
         -0.1813, -0.6304, -0.3573, -0.4688, -0.2797, -0.6681,  0.2858, -0.2367,
         -0.5968,  0.0105,  0.2524, -0.0919, -0.2754,  0.2188,  0.0174, -1.1037,
         -0.3949,  0.4523, -0.9401,  0.8944,  0.0947, -0.1656,  0.1163,  0.0230,
          0.0837, -0.5407, -0.0623,  0.1688, -0.0171,  0.2827,  0.4524, -0.3362,
         -0.2670, -0.4202, -1.0079, -0.4067, -1.0663, -0.4388, -0.0119, -0.7951,
         -0.9597,  0.0852,  0.7201,  0.1807,  0.5180, -0.5436, -0.0085, -0.4779,
         -0.1160, -0.4652, -0.1845, -0.1606, -0.1348, -1.0318, -0.4057,  0.3358,
          0.3188,  0.1393, -

          1.6605e+00,  1.9093e-01,  2.6044e-01,  1.6625e+00, -8.2073e-01]])
tensor([[ 0.1890,  1.0032,  0.9636, -2.0066, -1.6192, -0.7633, -1.1807,  1.4590,
          1.5294,  0.8068,  2.1906,  0.6450,  0.7056, -0.2617, -0.9709, -1.2463,
         -0.3092,  0.7252,  0.5662,  1.1134, -2.7689, -1.5565,  1.5494,  0.4242,
          1.6387, -0.5278,  0.3178, -1.4074, -0.7011, -2.2558, -0.2167,  0.6267,
          1.7273, -0.0329,  0.9666, -0.0770, -1.1288,  1.6993, -1.4370, -0.4305,
         -0.4342, -0.8910, -0.8264,  1.1121,  0.9256,  0.5472,  0.4626,  1.1794,
          0.8637,  0.9088, -0.5639,  1.0997, -0.0449,  2.6336, -0.4833, -1.2386,
         -0.4826, -1.0753,  0.3950,  0.9006, -1.1073, -1.0689,  0.3596,  0.0190,
         -0.4549, -0.5147,  1.8227, -3.0972,  0.5396,  0.6853,  0.7394,  0.4590,
          1.8104, -0.0674, -0.6214,  0.1797, -1.1974, -0.3041,  0.9076,  1.6602,
          0.0986, -1.8105,  2.4783,  0.3643, -0.0987, -1.8444, -0.2767,  2.5897,
          0.8275,  2.2038, -0.333

          6.1221e-01,  1.4644e+00,  4.5319e-01, -5.6898e-01, -5.3450e-02]])
tensor([[-2.9408e+00, -1.7017e+00, -6.2279e-01, -2.9816e+00,  2.2590e+00,
         -1.7671e+00,  2.0976e+00, -2.5399e+00, -1.9138e+00, -2.0250e+00,
         -2.9743e-01,  1.4056e+00,  6.7511e-02,  5.2533e-01, -1.3491e-01,
          1.3246e+00,  2.9547e-01, -6.7962e-01, -1.2098e+00,  1.6358e+00,
          1.8046e+00, -2.8776e-01, -2.6404e+00,  1.4916e+00,  1.1090e-01,
         -2.1224e+00,  1.3595e+00,  1.9836e+00, -6.5821e-01, -1.4586e+00,
         -1.1877e-01,  1.8801e+00,  3.5728e-01,  3.3426e+00, -1.1563e+00,
         -1.1959e+00, -2.0869e+00, -5.3899e-01,  1.2205e+00,  1.3775e+00,
          3.9650e+00, -1.2929e+00,  7.9866e-01, -9.3294e-01, -8.9394e-01,
         -9.5711e-01, -1.0996e-01, -1.9388e+00, -4.8610e-02, -1.2548e+00,
         -1.1527e+00,  1.5601e-01, -4.4745e-01,  2.3322e+00, -1.0240e+00,
          2.4460e+00,  1.8591e+00, -8.4548e-01,  8.8008e-01,  1.9833e+00,
          1.3727e+00, -3.3409e+00,  

          1.7057, -0.5965,  1.6106,  1.5595,  0.5085, -0.5035, -0.5167,  2.2240]])
tensor([[-0.2337, -1.5468, -2.2306, -0.5887,  1.2358,  0.3494,  0.1261, -0.3945,
         -3.4581, -2.3216,  0.1140,  0.7389,  1.6923,  0.4818,  0.3398, -0.9595,
         -0.6126,  1.0249, -1.5639,  0.1641, -0.8106,  1.4619,  0.1730,  0.9468,
          1.5522,  0.9135, -0.9078,  1.6638,  1.1468, -0.5466,  4.0002,  1.6553,
         -2.0930, -0.0798,  0.9532,  0.3004, -1.2083,  1.2912, -0.5109, -0.0669,
         -2.9641,  0.9979, -0.5663,  0.8918,  0.2803, -0.6476,  2.2112, -1.5374,
         -1.6121, -1.2295,  0.2476, -1.6223, -0.2281, -0.0737,  2.8242,  1.0059,
         -1.1831,  2.6074, -0.1255,  1.9661, -1.0561, -1.1744,  3.4550,  1.8500,
         -1.3954,  1.8634,  0.7679,  0.0049,  1.0461,  1.4125, -1.1947,  2.8580,
          0.3616, -0.0854,  1.0539, -1.7453,  0.6110, -0.9782,  0.8398, -3.0910,
         -0.1098,  1.0763, -0.4302,  1.8145, -1.6394, -0.9445, -0.2428, -1.8673,
          1.4462, -1.4406,

          2.7352e+00, -1.1881e+00,  1.3633e+00,  2.3337e+00, -1.9023e+00]])
tensor([[-4.1235, -1.0436, -0.7444, -0.9204,  0.1679,  0.5313,  0.1245,  0.3798,
          0.6385, -1.9685,  0.4174, -1.0915, -0.6459,  0.5971,  3.5570, -1.6091,
         -1.5838, -0.7654, -2.6103, -1.3719, -3.6354, -2.4884, -0.3781,  0.1744,
         -2.4837,  0.5041, -3.4662,  1.4925, -0.4728,  2.0160, -2.9829, -1.7203,
         -2.2205, -1.7304, -0.4253,  0.0218,  2.2384,  0.7917, -0.0637, -0.8999,
          2.7936, -0.2961, -1.8017,  1.3062, -3.1044, -1.3720, -1.3983,  1.6029,
         -2.2134,  1.1483, -1.0152, -0.1792, -0.6640, -0.7231, -0.0684,  1.0779,
         -2.2330,  0.2595,  4.9722,  1.4697, -0.1146, -3.4189,  0.8284,  1.7892,
          1.6608, -0.4041,  0.9582, -1.4305, -0.5687, -0.8937, -2.3118,  1.4994,
          1.7518, -1.8589, -1.8555,  1.8751, -0.3430,  0.0623, -1.5748,  1.2331,
         -2.5464, -3.2781, -2.5359, -0.6178, -3.1212,  0.2561, -0.7857, -0.8541,
         -0.4995, -1.0128,  0.183

...

         -1.2187,  0.0663, -1.2192, -0.4885, -1.5780, -1.4778, -3.3319,  0.2069]])
tensor([[-0.7346, -0.2282,  0.0565,  0.1710,  0.2765,  1.2090, -0.3513, -0.6672,
         -1.9466, -0.2279, -0.7529,  0.8475,  0.4725,  1.6239,  0.8723,  1.2300,
         -2.2199, -0.5984, -1.5664,  0.1767,  1.8046, -1.2682,  2.3904, -1.5110,
         -0.1037,  0.6917, -1.0461, -2.2288, -0.8007, -0.6639, -1.0231,  1.4154,
          0.1627,  0.3202, -1.1209,  0.6688, -0.6582, -1.5136, -0.8431, -0.6890,
          0.9629, -0.3697, -0.8328, -0.2627,  0.7298, -1.1009,  1.6532, -1.2063,
          1.6646, -0.8653, -0.5338, -0.3655,  0.9769, -2.3895,  0.6331,  0.8825,
          0.4626,  0.1096, -2.9015,  0.4857,  0.7258,  0.9081,  0.4904,  0.8578,
         -0.9975,  0.3554,  1.3861, -0.9600,  0.8937,  1.0453,  0.6129,  1.0149,
          0.0845,  1.3789,  1.3874,  0.6408,  1.5991, -1.3055,  0.8816,  1.4419,
         -0.8430, -0.9181,  1.2074,  0.8505, -0.4439, -0.8421,  0.6059,  0.0834,
         -0.7247,  1.5237,

tensor([[-8.2020e-01, -9.5554e-01,  5.8835e-02,  5.9595e-01,  5.3517e-01,
         -6.1228e-02,  8.6220e-01,  2.6080e-01, -5.4741e-01, -1.2517e-01,
         -2.7091e+00,  1.6686e+00, -1.4205e+00, -2.2787e+00,  3.5961e-01,
          1.9606e+00,  7.6822e-01,  1.5810e+00,  2.8180e-01, -2.3807e+00,
          1.0548e+00,  1.9199e+00,  1.4079e+00, -3.3564e+00,  8.8406e-01,
          1.0260e+00,  1.5677e+00, -3.0521e+00,  1.6328e+00, -1.4783e+00,
         -1.6080e+00, -1.4190e-01, -1.4973e+00,  7.9919e-01, -1.7722e+00,
          8.0320e-01,  1.1710e+00, -9.3711e-01, -2.3393e-01, -3.1434e+00,
         -9.6679e-01, -7.2825e-01, -3.1255e+00,  1.0719e+00,  3.4879e+00,
         -5.5460e-01, -7.7550e-01, -1.2386e+00, -2.0210e+00, -2.4543e+00,
         -6.2056e-01, -8.0045e-01,  5.0612e-01,  5.4577e-01,  2.1870e-01,
         -1.4002e+00,  1.7883e+00, -1.5140e+00, -1.6415e+00,  2.3006e+00,
         -2.7521e-01, -2.0647e+00,  6.5039e-02,  3.3233e-01,  2.9031e+00,
          1.8947e+00, -5.1940e-01, -2.

tensor([[-0.0531,  1.0480, -1.4598, -0.4928, -0.6863, -1.2153, -0.3613,  1.3660,
          0.0158,  0.5011,  2.6551, -1.4974,  0.7919,  3.0056, -1.2009,  0.3045,
          0.6284, -3.2348,  0.1677, -0.2445, -0.7663, -3.5478, -0.7336,  0.7575,
         -0.1265,  0.1730,  1.7861,  1.7723, -1.4412, -1.1036, -0.0848, -0.8011,
          1.1600,  1.1375, -1.0335,  3.2606, -0.1850,  0.8129,  1.7522, -2.8369,
          0.9095,  1.5207, -1.8044,  0.9496,  3.4588, -0.3478, -2.1810, -0.9247,
         -0.8129, -0.9266,  0.0982, -2.1436, -1.4529, -0.2601,  3.1963,  2.0095,
         -1.7374,  0.9480,  0.7678,  0.4835, -0.3872, -1.7218,  2.9296, -0.6605,
         -1.4249, -0.6497,  0.5379,  0.4683, -2.2408, -3.5394,  1.1858, -0.0272,
          1.6757, -0.1163,  2.2711,  1.6707,  0.7396, -2.0509, -2.5590,  1.7740,
         -0.1616, -1.5795, -0.1609, -0.1861,  0.4408,  1.4034,  0.0096, -0.6588,
          0.4006, -0.8628, -1.8573, -1.4211, -2.5725,  0.5847, -0.9171,  0.4385,
         -1.5950,  2.3920, -

         -1.3937e+00, -3.9195e-01, -1.3163e+00,  8.5389e-02,  2.5796e+00]])
tensor([[ 3.5492e+00, -1.2910e+00,  1.5047e+00,  1.1891e+00,  2.2082e+00,
          7.1097e-01,  1.8249e+00, -6.8168e-01, -1.4944e+00,  2.1214e+00,
          2.1489e+00, -2.2686e-01,  1.9719e-01, -3.3910e+00,  2.0518e+00,
          1.9465e+00,  4.5395e-01,  1.2833e+00, -4.4101e-01, -3.1583e+00,
         -7.5164e-01, -1.6349e+00, -3.1585e-01,  3.3536e+00, -1.8039e+00,
          2.0208e+00,  1.1617e+00, -1.0602e+00,  4.9335e-02,  1.0553e+00,
          1.8250e+00,  9.8541e-01, -1.7467e+00, -2.0781e+00,  2.4182e+00,
         -8.5854e-01, -2.4388e-01,  9.7906e-01,  1.4413e+00, -8.4832e-01,
          4.8739e-01,  6.3590e-01, -8.0445e-02,  1.4844e+00,  9.2850e-01,
         -6.6203e-01, -2.0717e+00, -1.5779e-01,  7.1747e-01,  1.0715e+00,
         -6.7961e-01,  1.6065e+00, -1.7013e+00, -9.8070e-01, -2.0824e+00,
          1.5240e+00, -3.4324e-01, -9.4166e-01, -3.6086e-01,  2.8088e+00,
         -1.5392e+00, -2.3320e+00, -

tensor([[-0.3085, -1.2523, -3.0755,  0.1524,  0.7434,  3.6883, -2.8386,  2.2929,
          0.6681,  4.1726,  0.6443, -0.5948, -0.3506,  1.2735,  1.4247, -2.3682,
         -0.8013, -2.7794,  0.0332, -1.2129,  1.7604, -1.3042, -1.3887,  0.1126,
         -1.7538, -2.2561,  0.2515,  2.4013, -0.0908,  0.8347,  0.8170, -1.9771,
         -1.9449, -1.3506, -4.2938,  0.2780, -0.8790, -3.2336, -0.5310,  0.4949,
         -5.3409,  1.5521,  2.0113, -4.1695, -0.4490,  1.4450,  1.0023, -1.4364,
         -0.6230,  1.0155,  0.8188, -2.6262,  1.4541,  1.7365, -0.2397,  0.3938,
         -1.3671, -3.7376, -0.1869,  0.6155, -0.0334,  0.3574,  0.7747,  0.4846,
         -0.8830, -1.2221,  4.0402,  3.0770, -1.2370, -1.6754,  1.1098,  0.3345,
         -1.2070,  1.7969,  0.6006,  1.8762,  1.6452,  0.9618, -0.9519,  1.4141,
          1.5489,  1.8730,  1.2056,  2.0117,  2.0877,  2.3072,  0.9983, -0.0777,
          1.1676,  1.4973,  0.8062,  2.3141, -0.1654,  1.6179,  1.6257, -2.0403,
         -0.7330,  1.9582,  

In [205]:
best_performers

tensor([[ 9.2315,  7.7709,  7.8833,  ...,  7.9377,  7.3055,  8.1079],
        [ 9.7218,  9.5276,  9.3146,  ...,  8.9890,  8.3138,  8.2052],
        [10.3954,  9.0493,  9.0542,  ...,  8.6739,  8.4123,  9.0580],
        ...,
        [12.8120, 12.3960, 12.1742,  ..., 10.6394, 11.1695, 10.3386],
        [12.3022, 12.2804, 12.8497,  ...,  9.6569,  9.5674,  9.9907],
        [12.9382, 12.6913, 12.3818,  ..., 10.8169, 10.4931, 10.9597]])

# Algorithm 2: Shift subspace to best performing point and nearest neighbors

In [35]:
torch.sum((X - X[1,:])**2, dim = 1)

tensor([ 154.2373,    0.0000,  449.7277,  484.4374,  178.5435,  230.8107,
         812.1345,  163.3415,  293.8382,  172.5614,  101.8661,  181.8690,
         429.0275,  161.9487,  476.5090,  197.9370,  665.9557,  162.8734,
         166.9253,  561.1698,  249.8136,  279.5471,  593.1084,  120.2866,
         239.8529,  225.0260,  315.6014, 1734.5291,  234.7258,  608.0305,
         169.3669,  487.4423,  454.7876,  506.7448,  174.5904,  654.9512,
         161.8866,  277.4589,  111.6341,  372.9736,  367.6244,  138.7883,
         691.6191, 1243.8972,  273.9227,  246.6556,  539.1151,  890.1454,
         347.1640,  601.5153,  154.2643,  261.5225,  422.0530,  957.8129,
         601.9742,  200.9976,  124.3907,  182.8707,  180.9987,  161.3325,
         435.5078,  394.3779,  119.5502,  312.6648,  400.4779,  404.6498,
         422.9346,  133.0938,  398.1880,  154.0300,  463.5797,  551.8766,
         233.1824,  452.8594,  398.8275,   96.5291, 1331.2234,  166.6382,
         239.1755,  932.1789,  292.220

In [102]:
def learn_subspace_best_performer_neighbors(fn, low_dim, fn_dim, num_neighbors = 1, samples = 200, max_iter = 1000, trace = False):
    '''
    Transition matrix doesn't update, but mean point does to best found point
    '''
    
    best_found = 1e10
    A = torch.randn((low_dim, fn_dim))
    B = torch.randn((samples, fn_dim))
    first_run = True
    
    for i in range(max_iter):
        # Evaluate
        x = torch.randn((samples,low_dim))
        x[0,:] = torch.zeros(low_dim)
        X = x@A + B
        y = fn.evaluate(X)

        # Find best performing points (Alt: Find best performing and nearest neighbors)
        best_index = torch.topk(y, 1, largest = False).indices
        distances = torch.sum((X - X[1,:])**2, dim = 1)
        new_subspace_indices = torch.topk(distances, num_neighbors + 1, largest = False).indices
        
        best_performers = X[new_subspace_indices]
#         best_performers = X[best_index]

        # Update
        B = torch.mean(best_performers, dim = 0)
        A = torch.randn((low_dim, fn_dim))
#         A = torch.concat([best_performers, torch.randn((low_dim - num_neighbors - 1, fn_dim))], dim = 0)

        if min(y) < best_found:
            best_found = min(y)

        if i%int(max_iter/10) == 0:
            if trace:
                print(f"Min/max of A: {torch.min(A):.2f}/{torch.max(A):.2f}")
            print(f"Min/best found y is:{min(y):.2f}, {best_found:.2f}")

In [105]:
fn.evaluate(torch.zeros(200))

tensor([9.5367e-07])

In [112]:
learn_subspace_best_performer_neighbors(fn, low_dim = 5, fn_dim = 10, num_neighbors = 2)

AssertionError: 

In [82]:
low_dim = 10
fn_dim = 200
num_neighbors = 4
samples = 100

In [83]:
best_found = 1e10
A = torch.randn((low_dim, fn_dim))
B = torch.randn((samples, fn_dim))

In [90]:
# Evaluate
x = torch.randn((samples,low_dim))
x[0,:] = torch.zeros(low_dim)
X = x@A + B
y = fn.evaluate(X)

In [91]:
# Find best performing points (Alt: Find best performing and nearest neighbors)
best_index = torch.topk(y, 1, largest = False).indices
distances = torch.sum((X - X[1,:])**2, dim = 1)
new_subspace_indices = torch.topk(distances, num_neighbors + 1, largest = False).indices

best_performers = X[new_subspace_indices]

In [92]:
if min(y) < best_found:
    best_found = min(y)
    
print(f"Min/best found y is:{min(y):.2f}, {best_found:.2f}")

Min/best found y is:6.95, 5.35


In [97]:
y

tensor([ 8.5770, 12.5789,  9.9687,  9.4279, 10.1412, 12.1972,  9.3779, 10.1123,
        14.9713, 10.3808, 11.0832,  9.6199, 12.7220, 11.5108, 13.3569, 10.0762,
        16.2185,  8.1241, 10.8028, 12.5537, 12.0864, 10.5939, 15.9956, 11.0310,
        14.8415, 12.8547, 13.7441, 12.4490, 13.2308, 12.3240, 13.2514, 13.1192,
        13.7652,  9.5375, 11.5863, 12.4260, 12.7587, 13.6723, 10.9530, 14.1791,
        10.2957,  9.5505, 11.5616, 12.0520, 12.0447, 14.3482, 12.3085, 12.9885,
        10.2062, 12.1251, 11.9854, 13.9223, 13.2727, 12.3311,  9.4562, 11.3405,
        11.1112, 12.6900, 10.9091, 11.4100, 12.7988, 14.4959, 10.4736, 10.8847,
        17.0617, 11.3661, 11.1866, 11.9426, 10.1181, 13.3619, 13.6523, 11.5104,
         8.3132, 10.6438, 12.2887, 11.3128, 13.6445, 12.9186, 13.7364, 14.1738,
        12.5506, 13.1405, 13.8867, 11.5185, 13.1022,  9.1402, 13.0639, 13.3303,
        12.4670, 12.4748,  6.9454,  8.7430, 11.8518, 14.7393, 14.1471, 12.2286,
        13.4755, 13.7506, 11.6791, 12.69

In [89]:
B = torch.mean(best_performers, dim = 0)